In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image

In [2]:
train_dir = 'chest_xray/train'
val_dir = 'chest_xray/val'
test_dir = 'chest_xray/test'

In [3]:
# Pré-processamento dos dados
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
# Construção do modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilação do modelo
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=val_generator,
    validation_steps=50
)

# Avaliação do modelo
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print(f'Test accuracy: {test_acc}')

c:\Users\Thais\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


c:\Users\Thais\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.7484 - loss: 0.5094 - val_accuracy: 0.6875 - val_loss: 0.6093
Epoch 2/20


c:\Users\Thais\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 63s 631ms/step - accuracy: 0.9244 - loss: 0.1868 - val_accuracy: 0.6250 - val_loss: 0.9281
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.9491 - loss: 0.1317 - val_accuracy: 0.6875 - val_loss: 0.6479
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 64s 634ms/step - accuracy: 0.9745 - loss: 0.0758 - val_accuracy: 0.6875 - val_loss: 0.7517
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9627 - loss: 0.1025 - val_accuracy: 0.8125 - val_loss: 0.4532
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 63s 625ms/step - accuracy: 0.9599 - loss: 0.1018 - val_accuracy: 0.8750 - val_loss: 0.4185
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9734 - loss: 0.0741 - val_accuracy: 0.9375 - val_loss: 0.2307
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 64s 634ms/step - accuracy: 0.9738 - loss: 0.0742 - val_accuracy: 0.8125 - val_loss: 0.4090
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.9742 - loss: 0.0765 - val_accurac

In [6]:
# Salvando o modelo
model.save('pneumonia_detection_model.h5')

In [7]:
# Função para carregar e prever nova imagem
def predict_image(model, img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print('Pneumonia Detectada')
    else:
        print('Pneumonia Não Detectada')

# Exemplo de uso da função de predição
predict_image(model, 'chest_xray/test/NORMAL/IM-0001-0001.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Pneumonia Não Detectada
